## Imports

In [14]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Input

## Read in Dataset

In [3]:
dimension = 1000
batch_size = 4

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/michw/Documents/fyp/final/dataset',
    labels='inferred',
    label_mode="int",
    class_names=['2_contour', '3_contour'],
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(dimension, dimension),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='training'
)

Found 24 files belonging to 2 classes.
Using 22 files for training.


## Building Model

In [4]:
model = keras.Sequential()

model.add(Input(shape = (dimension, dimension, 3)))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1000, 1000, 32)    896       
                                                                 
 conv2d_1 (Conv2D)           (None, 998, 998, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 499, 499, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 499, 499, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 499, 499, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 497, 497, 64)      36928     
                                                        

## Compile and Fit

In [6]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(ds_train, epochs=5, verbose=1)

Epoch 1/5
6/6 [==============================] - 34s 5s/step - loss: 1727.4449 - accuracy: 0.6818
Epoch 2/5
6/6 [==============================] - 33s 5s/step - loss: 178.7585 - accuracy: 0.8182
Epoch 3/5
6/6 [==============================] - 34s 6s/step - loss: 37.9850 - accuracy: 0.5909
Epoch 4/5
6/6 [==============================] - 33s 5s/step - loss: 1.3299 - accuracy: 0.9091
Epoch 5/5
6/6 [==============================] - 33s 5s/step - loss: 0.1514 - accuracy: 0.9545


##  Saving Model

In [7]:
# model.save("19-04-22-1127", save_format='h5')

## Model Style Two

In [15]:
dimension = 1000
batch_size = 4

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .1)

test_datagen = ImageDataGenerator(validation_split = .1)

In [16]:
train_generator = train_datagen.flow_from_directory(
    'C:/Users/michw/Documents/fyp/final/dataset',
    target_size=(dimension, dimension),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training')

Found 22 images belonging to 2 classes.


In [17]:
validation_generator = test_datagen.flow_from_directory(
    'C:/Users/michw/Documents/fyp/final/dataset',
    target_size=(dimension, dimension),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation')

Found 2 images belonging to 2 classes.


In [18]:
from keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(monitor='accuracy', verbose=1)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_generator, epochs=5, validation_data=validation_generator, verbose=1, callbacks=[earlyStopping])

Epoch 1/5
6/6 [==============================] - 35s 6s/step - loss: 8.1181 - accuracy: 0.9091 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
6/6 [==============================] - 34s 6s/step - loss: 0.3313 - accuracy: 0.9545 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2: early stopping


## Saving Model Type Two

In [19]:
# model.save("19-04-22-1139", save_format='h5')

## Testing Model

In [12]:
# import cv2
# import numpy as np

# CATEGORIES = ['2_contour', '3_contour']

# def prepare(filepath):

#     img_size = 1000

#     img = cv2.imread(filepath)
#     original = img

#     # Image pre-processing to imrpove clarity.
#     kernel = np.ones((5, 5), np.uint8)
#     img = cv2.dilate(img, kernel, iterations=1)
#     img = cv2.GaussianBlur(img,(1,1),0)
#     kernel = np.ones((5,5),np.float32)/25
#     img = cv2.filter2D(img,-1,kernel)

#     # Building colour range.
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     col_range = cv2.inRange(hsv, (36, 25, 25), (69, 255,255))

#     # Finding all contours in processed image.
#     contours, hierarchy = cv2.findContours(col_range, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
#     # Initialise 2 largest contours.
#     select1 = (0, 0, 0, 0)

#     # Finding the largest contour.
#     for contour in contours:
#         x, y, w, h = cv2.boundingRect(contour)
#         contour_area = w * h
#         select1_area = select1[2] * select1[3]

#         if contour_area > select1_area:
#             select1 = x, y, w, h

#     x, y, w, h = select1

#     adjustment = 7

#     img_array = original[y+adjustment:y + (h-adjustment), x+adjustment:x + (w-adjustment)]

#     new_array = cv2.resize(img_array, (img_size, img_size))

#     return new_array.reshape(1, img_size, img_size, 3)

# input = [ "C:/Users/michw/Documents/fyp/original_ml/dataset/box_t2",
#          "C:/Users/michw/Documents/fyp/original_ml/dataset/box_t3",
#          "C:/Users/michw/Documents/fyp/main/dataset_black/1_contour",
#          "C:/Users/michw/Documents/fyp/main/dataset_black/2_contour"]

# import os

# for i in input:
#     print(i)
#     for file in sorted(os.listdir(str(i))):

#         prediction = model.predict([prepare(i + "/" + file)])
#         # print(prediction)
#         classes = np.argmax(prediction, axis = 1)
#         # print(classes[0])
#         print(CATEGORIES[classes[0]])

In [21]:
# import cv2
# import numpy as np

# CATEGORIES = ['2_contour', '3_contour']
# # CATEGORIES = ['one', 'two', 'three']

# def prepare(filepath):
#     IMG_SIZE = 1000
#     img_array = cv2.imread(filepath)

#     new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

#     return new_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)

# prediction = model.predict([prepare('C:/Users/michw/Documents/fyp/original_ml/00362.jpg')])
# classes = np.argmax(prediction, axis = 1)
# print(CATEGORIES[classes[0]])
# print(prediction)

# prediction = model.predict([prepare('C:/Users/michw/Documents/fyp/original_ml/00262.jpg')])
# classes = np.argmax(prediction, axis = 1)
# print(CATEGORIES[classes[0]])
# print(prediction)

# prediction = model.predict([prepare('C:/Users/michw/Documents/fyp/original_ml/00204.jpg')])
# classes = np.argmax(prediction, axis = 1)
# print(CATEGORIES[classes[0]])
# print(prediction)